In [1]:
import pandas as pd
import numpy as np
import torchvision as tv
import jieba
from gensim import models
from gensim.models import Word2Vec
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch
show = ToPILImage() # 可以把Tensor转成Image，方便可视化
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('../../data/df_sen_sub/train.csv')
data

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高
5,TVciHBPL5XmUxMEd,我开始就是荣放2.5 森林人2.5二选一 荣放主要是底盘质感不行 太硬 其次是...,价格,1,便宜
6,4lb9TsO5rjqUy1Bu,唉，这货的价格死硬死硬的，低配版优惠1万据说已经罕有了。,价格,-1,死硬
7,z98xV7MphZIF4EvO,价格的话只能说一般般吧，太仓前段时间定的比你便宜！,价格,0,一般
8,hoWrMnGTUvdsK3Dm,听过，价格太贵，但一直念念不忘,价格,-1,贵
9,r1WHgbI0ZKqMdELy,恭喜恭喜，这个优惠不错哦！,价格,1,优惠不错


In [3]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
len(data['content_id'].unique()), len(list(data['content_id']))
content_ids = data['content_id'].unique()
subjs = data['subject'].unique()
df = {}
text, subs, sentis = [], [], []
for i in range(len(content_ids)):
    if i % 100 == 0:print(i)
    dd = data[data['content_id'] == content_ids[i]]
    text.append(dd.loc[dd.index.tolist()[0], 'content'])
    temp_subs, temp_senti = [], []
    for sub in subjs:
        temp_subs.append(sub)
        if len(dd[dd['subject'] == sub]) != 0:
            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))
        else:
            temp_senti.append('2')
    subs.append(','.join(temp_subs))
    sentis.append(','.join(temp_senti))
df['content'] = text
df['subjects'] = subs
df['sentiments'] = sentis
df = pd.DataFrame(df)
df.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)
#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nlen(data['content_id'].unique()), len(list(data['content_id']))\ncontent_ids = data['content_id'].unique()\nsubjs = data['subject'].unique()\ndf = {}\ntext, subs, sentis = [], [], []\nfor i in range(len(content_ids)):\n    if i % 100 == 0:print(i)\n    dd = data[data['content_id'] == content_ids[i]]\n    text.append(dd.loc[dd.index.tolist()[0], 'content'])\n    temp_subs, temp_senti = [], []\n    for sub in subjs:\n        temp_subs.append(sub)\n        if len(dd[dd['subject'] == sub]) != 0:\n            temp_senti.append(str(int(dd[dd['subject'] == sub]['sentiment_value'])))\n        else:\n            temp_senti.append('2')\n    subs.append(','.join(temp_subs))\n    sentis.append(','.join(temp_senti))\ndf['content'] = text\ndf['subjects'] = subs\ndf['sentiments'] = sentis\ndf = pd.DataFrame(df)\ndf.to_csv('../../data/df_sen_sub/merge_train.csv', index = False)\n#data = pd.read_csv('../../data/df_sen_sub/test_public.csv')\n"

In [4]:
data = pd.read_csv('../../data/df_sen_sub/merge_train.csv')
data = data.fillna('')
subs = list(data['subjects'])
sentiments = list(data['sentiments'])
content = list(data['content'])
test_data = pd.read_csv('../../data/df_sen_sub/test_public.csv')
test_data = test_data.fillna('')
test_content = list(test_data['content'])
data = pd.read_csv('../../data/df_sen_sub/train.csv')
data = data.fillna('')
senti_words = list(data['sentiment_word'])

In [5]:
with open('../../data/df_sen_sub/add_text.txt','w') as f:
    f.write('\n'.join(list(content) + list(test_content)  + list(subs) + senti_words))
# 迭代器，使用jieba将句子进行分词
stopwords = open('stopwords.txt', 'r').read().split('\n')
class Sentences(object):# 这个类可以根据实际情况重写，我已经将所有的文章进行分句，并整合到了一个文件里面
    def __init__(self, dirname):
        self.dirname = dirname # 句子所在文件，没句句子占一行
        #jieba.load_userdict("wordBase.txt") # 加载词库

    def __iter__(self):
        #for fname in os.listdir(self.dirname):
        for line in open(self.dirname):
                line = line.replace('\n', '')
                res = []
                for word in list(jieba.cut(line)):
                    if word in stopwords:
                        continue
                    res.append(word)
                yield res#list(jieba.cut(line))

sentences = []
def train_word2vec(folder_path, size=100):
    global sentences
    sentences = Sentences(folder_path) #生成分词后的句子，是一个二维数组

    # size是词向量长度
    # worker是线程数量，建议与物理线程数量一致
    # min_count是指出现次数小于一定程度，就忽略，0表示不忽略
    #model = Word2Vec(sentences, size=size, workers=8, min_count=0)

    # 训练结束就将模型保存起来
    #model.save("../../data/df_sen_sub/add_word2vec_model")

# 生成50维度的词向量模型
train_word2vec("../../data/df_sen_sub/add_text.txt",100)

# 测试训练好的词向量模型，使用model[keyWord]即可获取keyword这个词的词向量
model = Word2Vec.load("../../data/df_sen_sub/add_word2vec_model")
sentences = list(sentences)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.059 seconds.
Prefix dict has been built succesfully.


In [6]:
len(sentences)

28890

In [7]:
from collections import Counter
data = pd.read_csv('../../data/df_sen_sub/train.csv')
sub_uniq = data['subject'].unique()
data = data[pd.notnull(data['sentiment_word'])]
senti_word_set = {}
sum_ = 0
for key in sub_uniq:
    co = 0
    dd = list(data[data['subject'] == key]['sentiment_word'])
    ss = []
    for phras in dd:
        words = list(jieba.cut(phras))
        for word in words:
            word = word.replace(' ', '')
            if word == '' or word in stopwords:continue
            ss.append(word)
            co += 1
        counter = Counter(ss)
        sort = sorted(counter.items(), key=lambda item: item[1], reverse = True)
    senti_word_set[key] = [sort[kk][0] for kk in range(len(sort))]
    print(key, co)
    sum_ += co
print(sum_)
senti_word_set

价格 262
配置 266
操控 359
舒适性 386
油耗 272
动力 858
内饰 280
安全性 94
空间 162
外观 262
3201


{'价格': ['便宜',
  '优惠',
  '贵',
  '价格',
  '高',
  '低',
  '不错',
  '差不多',
  '合适',
  '性价比',
  '不会',
  '太贵',
  '不高',
  '没有',
  '好',
  '值',
  '合理',
  '不能',
  '行',
  '小贵',
  '没',
  '不菲',
  '不要',
  '一点',
  '实惠',
  '影响',
  '有钱',
  '任性',
  '死硬',
  '幅度',
  '不小',
  '后悔',
  '太小',
  '套路',
  '不值',
  '愉快',
  '高昂',
  '下不来',
  '亲民点',
  '放弃',
  '不够',
  '平易近人',
  '老贵',
  '保证',
  '好多',
  '差距',
  '更',
  '知道',
  '比较',
  '就行',
  '那点',
  '行情',
  '价',
  '正常',
  '几百',
  '参差不齐',
  '贵太多',
  '买',
  '挺高',
  '价差',
  '很大',
  '在意',
  '下降',
  '奇高',
  '不好',
  '很多',
  '超级',
  '过高',
  '售价',
  '明显',
  '高于',
  '竞品',
  '最好',
  '操控',
  '满意',
  '优势',
  '费用',
  '虚高',
  '相当',
  '不算',
  '再贵',
  '车',
  '欧蓝德',
  '真心',
  '秒杀',
  '对手',
  '单价',
  '降价',
  '想',
  '力度',
  '保值',
  '率',
  '大众',
  '略贵',
  '还行',
  '保养',
  '良心',
  '找不出',
  '第二辆',
  '不用',
  '考虑',
  '不靠',
  '谱',
  '没法',
  '咬咬牙',
  '不合算'],
 '配置': ['导航',
  '高',
  '好',
  '不错',
  '手机',
  '没有',
  '垃圾',
  '没',
  '差',
  '喜欢',
  '低',
  '配置',
  '实用',
  '鸡肋',
  '一点',
  '提升',
  '比较',
  '欧蓝德',

In [8]:
x_vecs = []
x_test_vecs = []  
ind = 0
word2ind = {}
pretrained = []
senti_sen = []
senti_test_sen = []
for i in range(len(test_content) + len(content)):
    if i < len(content):
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_vecs.append(temp)
        senti_sen.append(t_s[:50])
    else:
        temp = []
        t_s = []
        for j in range(len(sentences[i])):
            if sentences[i][j] not in word2ind:
                pretrained.append(model[sentences[i][j]])
                word2ind[sentences[i][j]] = ind
                ind += 1
            t_s.append(sentences[i][j])
            temp.append(model[sentences[i][j]])
        while len(temp) < 50:
            temp.append([0.0] * 100)
        if len(temp) > 50:
            temp = temp[:50]
        x_test_vecs.append(temp)
        senti_test_sen.append(t_s[:50])     
x_vecs = np.array(x_vecs)
x_test_vecs = np.array(x_test_vecs)
print(x_vecs.shape, x_test_vecs.shape)

/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(8290, 50, 100) (2364, 50, 100)


In [9]:
x_sent_vecs = {}
x_test_sent_vecs = {}
for key in senti_word_set:
    x_sent_vecs[key] = []
    x_test_sent_vecs[key] = []
    for i in range(len(content) + len(test_content)):
        if i < len(content):
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(1)
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_sent_vecs[key].append(t_sent)
        else:
            t_sent = []
            for j in range(len(sentences[i])):
                if sentences[i][j] in senti_word_set[key]:
                    t_sent.append(1)
                else:
                    t_sent.append(0) 
            while len(t_sent) < 50:
                t_sent.append(0)
            if len(t_sent) > 50:
                t_sent = t_sent[:50]
            x_test_sent_vecs[key].append(t_sent)    

In [10]:
for key in x_test_sent_vecs:
    print(key,len(x_sent_vecs['价格']))

价格 8290
配置 8290
操控 8290
舒适性 8290
油耗 8290
动力 8290
内饰 8290
安全性 8290
空间 8290
外观 8290


In [11]:
#va, content_vecs, y
pretrained.append([0.0] * 100)
word2ind['null'] = 17387
map_ = ['-1', '0', '1', '2']
sub_set = {}
for i in range(len(subs)):
    subjs = subs[i].split(',')
    sentis = sentiments[i].split(',')
    for j in range(len(subjs)):
        if subjs[j] not in sub_set:
            sub_set[subjs[j]] = []
        sub_set[subjs[j]].append(map_.index(sentis[j]))

In [12]:
pretrained = np.array(pretrained)
han_x = []
for i in range(len(senti_sen)):
    temp = []
    for j in range(len(senti_sen[i])):
        temp.append(word2ind[senti_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x.append(temp)
han_x = np.array(han_x)

han_x_test = []
for i in range(len(senti_test_sen)):
    temp = []
    for j in range(len(senti_test_sen[i])):
        temp.append(word2ind[senti_test_sen[i][j]])
    while len(temp) < 50:
        temp.append(17387)

    temp = temp[:50]

    han_x_test.append(temp)
han_x = np.array(han_x)
han_x_test = np.array(han_x_test)

In [43]:

'''
class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True) 
        #self.fc1   = nn.Linear(3750, 50) 
        #self.fc2   = nn.Linear(200, 50)
        #self.fc3   = nn.Linear(200, 50)
        #self.fc4   = nn.Linear(100, 3)
        self.fctest   = nn.Linear(200, 3)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, aspects): 
        x = self.embeddings(x)#batch,50,100
        #va = self.embeddings(aspects[0])
        #t_va = va.unsqueeze(0).expand(x.shape[0], x.shape[1], 100)
        output, (hn, cn) =  self.bilstm(x.float())#batch,50,200
        H, hN = output, output[:,-1,:]
        #con = torch.cat((t_va, H), 2)
        #t1 = F.max_pool2d(F.relu(con), (1, 4))
        #con = t1.view(x.size()[0], -1)
        #con = F.relu(self.fc1(con))
        #alpha = con.unsqueeze(1)
        #rr = torch.matmul(alpha,H).squeeze(1)
        #yy = torch.cat((F.relu(self.fc2(rr)), F.relu(self.fc3(hN))), 1)
        #x = self.fc4(yy)
        x  = self.fctest(hN)
        return x
    

net = Net(pretrained, len(pretrained))
print(net)
from torch import optim
criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
'''
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, pretrained, vocab_size):
        super(Net, self).__init__()
        #self.rnn1 = nn.RNN(100, 100) 
        self.bilstm = nn.LSTM(input_size = 100, hidden_size = 100, bidirectional = True)
        #self.fc1   = nn.Linear(1250, 120) 
        self.fch = nn.Linear(200, 100)
        self.fcv = nn.Linear(200, 100)
        self.fcw = nn.Linear(200, 1)
        self.fcwp = nn.Linear(200,50)
        self.fcwh = nn.Linear(200,50)
        self.fcwsen = nn.Linear(200, 50)
        self.fc1   = nn.Linear(3750, 120) 
        self.fc2   = nn.Linear(120, 50)
        self.fc3   = nn.Linear(50, 4)
        self.embeddings = nn.Embedding(vocab_size, 100)
        pretrained_weight = np.array(pretrained)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weight))
        
    def forward(self, x, x_sent, aspects): 
        x = self.embeddings(x) #[4,50,100]
        batch_size = x.shape[0]
        N = x.shape[1]
        output, (hn, cn) =  self.bilstm(x.float())  #[4,50,200]
        hN = output[:,-1,:]
        va = self.embeddings(aspects[0]) #[1, 100]
        t_va = torch.matmul(torch.ones((x.shape[0],50, 1)), va)#[4, 50, 100]
        x_sent = x_sent.float()
        #[4, 50] *? = [4, 50, 100]
        sent_va = torch.mul(torch.matmul(x_sent.reshape((x.shape[0], 50, 1)), torch.ones((x.shape[0],1, 100))), t_va)  #[4, 50, 100]
        t_va = torch.cat((sent_va, t_va),  2)
        t_va = t_va.reshape((-1, 200)) #[4*50, 100]
        x = output.reshape((-1, 200)) #[4*50, 200]
        x = torch.tanh(torch.cat((self.fch(x), self.fcv(t_va)), 1))
        M = x.reshape((batch_size, N, -1)) #[4, 50, 200]
        alpha = F.softmax(self.fcw(M)).reshape((batch_size, 1, -1))#[4,50]
        rr = torch.matmul(alpha, output).reshape((batch_size, -1)) #[4, 200]
        x_sent = x_sent.reshape((batch_size, 1, -1))
        ss = torch.matmul(x_sent.float(), output).reshape((batch_size, -1))
        final = torch.tanh(self.fcwh(hN) + self.fcwp(rr) + self.fcwsen(ss))
        x = self.fc3(final)   
        return x, alpha
    


In [44]:
ii = 0
import time
for key in sub_set:
    if ii == 10:
        break
    ii += 1
    net = Net(pretrained, len(pretrained))
    cur_max_acc = 0.0
    print(net)
    from torch import optim
    criterion = nn.CrossEntropyLoss() # 交叉熵损失函数
    optimizer = optim.Adagrad(net.parameters(), lr=0.01)#, momentum=0.9)
    train_x, test_x, train_y, test_y = train_test_split(han_x,sub_set[key], test_size=0.01, random_state=42)
    train_x_sents, test_x_sents, train_y_, test_y_ = train_test_split(np.array(x_sent_vecs[key]),sub_set[key], test_size=0.01, random_state=42)
    trainset = [((train_x[i],train_x_sents[i]), train_y[i]) for i in range(len(train_x))]
    testset = [((test_x[i],test_x_sents[i]), test_y[i]) for i in range(len(test_x))]
    trainloader = torch.utils.data.DataLoader(
                        trainset, 
                        batch_size=4,
                        shuffle=True, 
                        num_workers=2)
    testloader = torch.utils.data.DataLoader(
                        testset, 
                        batch_size=4,
                        shuffle=False, 
                        num_workers=2)
    torch.set_num_threads(8)
    for epoch in range(20):  

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):

            # 输入数据
            (inputs, sents), labels = data
            # 梯度清零
            optimizer.zero_grad()
            # forward + backward 
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            outputs,alpha = net(inputs, sents, va)
            #plt.imshow(alpha.detach().numpy()[0])
            
            #plt.colorbar()
            #plt.show()  
            loss = criterion(outputs, labels)
            loss.backward()   

            # 更新参数 
            optimizer.step()

            # 打印log信息
            # loss 是一个scalar,需要使用loss.item()来获取数值，不能使用loss[0]
            running_loss += loss.item()
            if i % 200 == 199: # 每200个batch打印一下训练状态
                print('[%d, %5d] loss: %.3f' \
                      % (epoch+1, i+1, running_loss / 200))
                running_loss = 0.0
                
        with torch.no_grad():
            correct, total = 0, 0
            for data in testloader:
                (images, sentis), labels = data
                va = [[word2ind[key]]]
                va = torch.from_numpy(np.array(va))
                outputs, alpha = net(images,sentis, va)
                _, predicted = torch.max(outputs, 1)
                #import ipdb;ipdb.set_trace()
                #res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
                total += labels.size(0)
                correct += (predicted == labels).sum()
            if cur_max_acc < float(100 * correct / total):
                print('-------------cur max is ::::----------'+str(float(100 * correct / total)))
                cur_max_acc = float(100 * correct / total)
                torch.save(net, 'net+' + key + '.pkl')  # 保存整个网络
            print('result of epoch ' + str(epoch) + 'accuracy is : %d %%' % (100 * correct / total))

    print('key' + key)
    #net2 = torch.load('net.pkl')
    #prediction = net2(x)
    print('Finished Training')


Net(
  (bilstm): LSTM(100, 100, bidirectional=True)
  (fch): Linear(in_features=200, out_features=100, bias=True)
  (fcv): Linear(in_features=200, out_features=100, bias=True)
  (fcw): Linear(in_features=200, out_features=1, bias=True)
  (fcwp): Linear(in_features=200, out_features=50, bias=True)
  (fcwh): Linear(in_features=200, out_features=50, bias=True)
  (fcwsen): Linear(in_features=200, out_features=50, bias=True)
  (fc1): Linear(in_features=3750, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=4, bias=True)
  (embeddings): Embedding(19307, 100)
)


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   200] loss: 0.423
[1,   400] loss: 0.207
[1,   600] loss: 0.252
[1,   800] loss: 0.221
[1,  1000] loss: 0.265
[1,  1200] loss: 0.252
[1,  1400] loss: 0.203
[1,  1600] loss: 0.246
[1,  1800] loss: 0.199
[1,  2000] loss: 0.213
-------------cur max is ::::----------95.0
result of epoch 0accuracy is : 95 %


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[2,   200] loss: 0.229
[2,   400] loss: 0.168
[2,   600] loss: 0.196
[2,   800] loss: 0.204
[2,  1000] loss: 0.211
[2,  1200] loss: 0.187
[2,  1400] loss: 0.197
[2,  1600] loss: 0.182
[2,  1800] loss: 0.169
[2,  2000] loss: 0.209
result of epoch 1accuracy is : 95 %
[3,   200] loss: 0.115
[3,   400] loss: 0.171
[3,   600] loss: 0.206
[3,   800] loss: 0.150
[3,  1000] loss: 0.188
[3,  1200] loss: 0.197
[3,  1400] loss: 0.190
[3,  1600] loss: 0.181
[3,  1800] loss: 0.190
[3,  2000] loss: 0.185
result of epoch 2accuracy is : 95 %
[4,   200] loss: 0.195
[4,   400] loss: 0.188
[4,   600] loss: 0.153
[4,   800] loss: 0.157
[4,  1000] loss: 0.122
[4,  1200] loss: 0.148
[4,  1400] loss: 0.178
[4,  1600] loss: 0.121
[4,  1800] loss: 0.180
[4,  2000] loss: 0.146
result of epoch 3accuracy is : 95 %
[5,   200] loss: 0.147
[5,   400] loss: 0.157
[5,   600] loss: 0.167
[5,   800] loss: 0.155
[5,  1000] loss: 0.153
[5,  1200] loss: 0.138
[5,  1400] loss: 0.165
[5,  1600] loss: 0.123
[5,  1800] loss: 0

...

[1,   200] loss: 0.358
[1,   400] loss: 0.208
[1,   600] loss: 0.195
[1,   800] loss: 0.179
[1,  1000] loss: 0.152
[1,  1200] loss: 0.213
[1,  1400] loss: 0.167
[1,  1600] loss: 0.209
[1,  1800] loss: 0.198
[1,  2000] loss: 0.168
-------------cur max is ::::----------98.0
result of epoch 0accuracy is : 98 %
[2,   200] loss: 0.171
[2,   400] loss: 0.164
[2,   600] loss: 0.142
[2,   800] loss: 0.174
[2,  1000] loss: 0.170
[2,  1200] loss: 0.138
[2,  1400] loss: 0.202
[2,  1600] loss: 0.155
[2,  1800] loss: 0.132
[2,  2000] loss: 0.156
-------------cur max is ::::----------100.0
result of epoch 1accuracy is : 100 %
[3,   200] loss: 0.135
[3,   400] loss: 0.133
[3,   600] loss: 0.173
[3,   800] loss: 0.162
[3,  1000] loss: 0.165
[3,  1200] loss: 0.150
[3,  1400] loss: 0.167
[3,  1600] loss: 0.154
[3,  1800] loss: 0.137
[3,  2000] loss: 0.129
result of epoch 2accuracy is : 100 %
[4,   200] loss: 0.145
[4,   400] loss: 0.124
[4,   600] loss: 0.119
[4,   800] loss: 0.156
[4,  1000] loss: 0.12

[8,  1000] loss: 0.271
[8,  1200] loss: 0.232
[8,  1400] loss: 0.228
[8,  1600] loss: 0.255
[8,  1800] loss: 0.281
[8,  2000] loss: 0.253
result of epoch 7accuracy is : 79 %
[9,   200] loss: 0.271
[9,   400] loss: 0.232
[9,   600] loss: 0.239
[9,   800] loss: 0.276
[9,  1000] loss: 0.201
[9,  1200] loss: 0.200
[9,  1400] loss: 0.237
[9,  1600] loss: 0.270
[9,  1800] loss: 0.193
[9,  2000] loss: 0.240
result of epoch 8accuracy is : 77 %
[10,   200] loss: 0.233
[10,   400] loss: 0.276
[10,   600] loss: 0.173
[10,   800] loss: 0.200
[10,  1000] loss: 0.184
[10,  1200] loss: 0.213
[10,  1400] loss: 0.180
[10,  1600] loss: 0.211
[10,  1800] loss: 0.202
[10,  2000] loss: 0.214
result of epoch 9accuracy is : 78 %
[11,   200] loss: 0.196
[11,   400] loss: 0.169
[11,   600] loss: 0.169
[11,   800] loss: 0.196
[11,  1000] loss: 0.190
[11,  1200] loss: 0.192
[11,  1400] loss: 0.187
[11,  1600] loss: 0.178
[11,  1800] loss: 0.197
[11,  2000] loss: 0.167
result of epoch 10accuracy is : 78 %
[12,   

[15,  1800] loss: 0.014
[15,  2000] loss: 0.029
result of epoch 14accuracy is : 97 %
[16,   200] loss: 0.020
[16,   400] loss: 0.020
[16,   600] loss: 0.025
[16,   800] loss: 0.014
[16,  1000] loss: 0.019
[16,  1200] loss: 0.022
[16,  1400] loss: 0.008
[16,  1600] loss: 0.009
[16,  1800] loss: 0.010
[16,  2000] loss: 0.022
result of epoch 15accuracy is : 96 %
[17,   200] loss: 0.009
[17,   400] loss: 0.006
[17,   600] loss: 0.017
[17,   800] loss: 0.014
[17,  1000] loss: 0.018
[17,  1200] loss: 0.017
[17,  1400] loss: 0.026
[17,  1600] loss: 0.016
[17,  1800] loss: 0.011
[17,  2000] loss: 0.017
result of epoch 16accuracy is : 97 %
[18,   200] loss: 0.012
[18,   400] loss: 0.012
[18,   600] loss: 0.013
[18,   800] loss: 0.007
[18,  1000] loss: 0.015
[18,  1200] loss: 0.014
[18,  1400] loss: 0.013
[18,  1600] loss: 0.016
[18,  1800] loss: 0.011
[18,  2000] loss: 0.019
result of epoch 17accuracy is : 97 %
[19,   200] loss: 0.014
[19,   400] loss: 0.013
[19,   600] loss: 0.008
[19,   800] 

[1,   200] loss: 0.285
[1,   400] loss: 0.150
[1,   600] loss: 0.119
[1,   800] loss: 0.103
[1,  1000] loss: 0.126
[1,  1200] loss: 0.156
[1,  1400] loss: 0.093
[1,  1600] loss: 0.107
[1,  1800] loss: 0.098
[1,  2000] loss: 0.117
-------------cur max is ::::----------96.0
result of epoch 0accuracy is : 96 %
[2,   200] loss: 0.087
[2,   400] loss: 0.094
[2,   600] loss: 0.108
[2,   800] loss: 0.115
[2,  1000] loss: 0.092
[2,  1200] loss: 0.110
[2,  1400] loss: 0.096
[2,  1600] loss: 0.075
[2,  1800] loss: 0.112
[2,  2000] loss: 0.100
result of epoch 1accuracy is : 96 %
[3,   200] loss: 0.091
[3,   400] loss: 0.105
[3,   600] loss: 0.071
[3,   800] loss: 0.062
[3,  1000] loss: 0.100
[3,  1200] loss: 0.092
[3,  1400] loss: 0.099
[3,  1600] loss: 0.118
[3,  1800] loss: 0.089
[3,  2000] loss: 0.093
result of epoch 2accuracy is : 96 %
[4,   200] loss: 0.072
[4,   400] loss: 0.098
[4,   600] loss: 0.112
[4,   800] loss: 0.102
[4,  1000] loss: 0.070
[4,  1200] loss: 0.079
[4,  1400] loss: 0.07

[8,  1000] loss: 0.071
[8,  1200] loss: 0.061
[8,  1400] loss: 0.056
[8,  1600] loss: 0.077
[8,  1800] loss: 0.040
[8,  2000] loss: 0.066
result of epoch 7accuracy is : 95 %
[9,   200] loss: 0.056
[9,   400] loss: 0.064
[9,   600] loss: 0.048
[9,   800] loss: 0.055
[9,  1000] loss: 0.052
[9,  1200] loss: 0.048
[9,  1400] loss: 0.062
[9,  1600] loss: 0.055
[9,  1800] loss: 0.056
[9,  2000] loss: 0.042
result of epoch 8accuracy is : 95 %
[10,   200] loss: 0.058
[10,   400] loss: 0.049
[10,   600] loss: 0.047
[10,   800] loss: 0.049
[10,  1000] loss: 0.060
[10,  1200] loss: 0.057
[10,  1400] loss: 0.030
[10,  1600] loss: 0.049
[10,  1800] loss: 0.067
[10,  2000] loss: 0.049
result of epoch 9accuracy is : 92 %
[11,   200] loss: 0.045
[11,   400] loss: 0.040
[11,   600] loss: 0.039
[11,   800] loss: 0.054
[11,  1000] loss: 0.060
[11,  1200] loss: 0.063
[11,  1400] loss: 0.047
[11,  1600] loss: 0.049
[11,  1800] loss: 0.033
[11,  2000] loss: 0.041
result of epoch 10accuracy is : 95 %
[12,   

In [45]:
len(han_x_test),len(x_test_sent_vecs)

(2364, 10)

In [46]:
import torch.nn as nn
import torch.nn.functional as F
result_set = {}
for key in sub_set:
    print(key)
    res = []
    with torch.no_grad():
        correct, total = 0, 0
        testset = [(han_x_test[i],np.array(x_test_sent_vecs[key][i])) for i in range(len(han_x_test))]
        testloader = torch.utils.data.DataLoader(
                            testset, 
                            batch_size=4,
                            shuffle=False, 
                            num_workers=2)
        for data in testloader:
            # 梯度清零
            optimizer.zero_grad()
            # forward + backward 
            (inputs, sentis) = data
            va = [[word2ind[key]]]
            va = torch.from_numpy(np.array(va))
            net = torch.load('net+' + key + '.pkl')
            outputs,alpha = net(inputs, sentis, va)
            _, predicted = torch.max(outputs, 1)
            res = res + [map_[int(predicted[i])] for i in range(len(predicted))]
    result_set[key] = res


价格


/home/works/dl-tools/anaconda2/envs/yxvenv/lib/python3.6/site-packages/ipykernel/__main__.py:78: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


配置
操控
舒适性
油耗
动力
内饰
安全性
空间
外观


In [47]:
'''
data = pd.read_csv('../../data/df_sen_sub/train.csv')
uni_sub = data['subject'].unique()
for key in uni_sub:
    print(key + str(len(data[data['subject'] == key])))
    ttt = data[data['subject'] == key]
    print(len(ttt[ttt['sentiment_value'] == 0]))
    print(len(ttt[ttt['sentiment_value'] == -1]))
    print(len(ttt[ttt['sentiment_value'] == 1]))
'''

"\ndata = pd.read_csv('../../data/df_sen_sub/train.csv')\nuni_sub = data['subject'].unique()\nfor key in uni_sub:\n    print(key + str(len(data[data['subject'] == key])))\n    ttt = data[data['subject'] == key]\n    print(len(ttt[ttt['sentiment_value'] == 0]))\n    print(len(ttt[ttt['sentiment_value'] == -1]))\n    print(len(ttt[ttt['sentiment_value'] == 1]))\n"

In [48]:
real_pred = {}
content_ids = []
subjects = []
sentiment_values = []
sentiment_words = []
contents = []
#content_id,subject,sentiment_value,sentiment_word
for i in range(len(test_data)):
    content_id = test_data.loc[i, 'content_id']
    flag = 0
    for key in result_set:
        if result_set[key][i] != '2':
            flag = 1
            content_ids.append(str(content_id))
            subjects.append(key)
            sentiment_values.append(int(result_set[key][i]))
            sentiment_words.append("")
            contents.append(str(test_data.loc[i, 'content']))
    if flag == 0:
        content_ids.append(str(content_id))
        subjects.append('动力')
        sentiment_values.append(0)
        sentiment_words.append("")   
        contents.append(str(test_data.loc[i, 'content']))
real_pred = {"content_id":content_ids, "subject":subjects, "sentiment_value":sentiment_values, "sentiment_word":sentiment_words}
real_df = pd.DataFrame(real_pred)
real_df.to_csv('final_result+.csv', encoding = "utf-8", index = False)

In [49]:
len(real_df), len(test_data)

(2661, 2364)

In [50]:
real_pred['content'] = contents

dtt = pd.DataFrame(real_pred)
dtt.to_csv('see_final_result+.csv', encoding = "utf-8", index = False)

In [51]:
len(dtt[dtt['sentiment_value'] == 1])

116